%%
% Project Name: USSP
% Description: main function of experiment 6.3, the CNN in NICO 
% Author: Yang Jinjing
% Email: yangjinjing94@163.com
% Date: 2025-04-19
%%

In [ ]:
# Generate the uniform design
import numpy as np 
import pyunidoe as pydoe
uni_design=pydoe.gen_ud(n=2000, s=20, q=2000, init="rand", crit="CD2", maxiter=50, vis=False)
print(np.shape(uni_design["final_design"]))
np.save('UD_2000_20.npy', uni_design["final_design"])

In [ ]:
# Generate the LHD design
from pyDOE import *
lhd_design=lhs(20, samples=2000, criterion='maximin')
print(np.shape(lhd_design))
print(lhd_design)
np.save('LHD_2000_20.npy', lhd_design)

In [ ]:
## Load the package
import os
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import datasets, transforms, models
from PIL import Image
from tqdm import tqdm
from importlib import reload
import pickle
import NICO_transforms
reload(NICO_transforms)
import torch.nn.functional as F
import random

import USSP
reload(USSP)
import IBOSS
import LowCon
reload(LowCon)

import NICO_ResNet50_TrainTest
reload(NICO_ResNet50_TrainTest)

class DatasetFromList(Dataset):
    def __init__(self, data):
        self.data = data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx]

In [ ]:
## Enhance the Autumn dataset
# load the original dataset
train_dir = "../realdata/NICO/public_dg_0416/train/autumn"
original_dataset = datasets.ImageFolder(root=train_dir)


# enhance the dataset
augmented_data = []
for image, label in original_dataset:
    transformed_images = NICO_transforms.NICO_transforms(image)
    for transformed_image in transformed_images:
        augmented_data.append((transformed_image, label))

full_train_set = DatasetFromList(augmented_data)
print(f"Length of the enhanced file: {len(full_train_set)}") 



filename = "NICO_autumn_enhanced_dataset.pkl"

with open(filename, 'wb') as f:
    pickle.dump(full_train_set, f)

print(f"Saved to file: {filename}")

In [ ]:
# Load the enhanced autumn file
with open("NICO_autumn_enhanced_dataset.pkl", "rb") as f:
    full_train_set = pickle.load(f)


In [ ]:
## Enhance the test dataset and save to pkl

class AddGaussianNoise(object):
    def __init__(self, mean=0., std=1.):
        self.std = std
        self.mean = mean

    def __call__(self, tensor):
        torch.manual_seed(11)
        return tensor + torch.randn(tensor.size()) * self.std + self.mean

    def __repr__(self):
        return self.__class__.__name__ + '(mean={0}, std={1})'.format(self.mean, self.std)

def preprocess_test_data(test_dirs, output_dir="preprocessed_test_data"):
    """
    Preprocesses test datasets by applying transformations and saving them as pickle files.

    Args:
        test_dirs (list): A list of paths to the directories containing the test scenarios.
        output_dir (str): The directory to save the preprocessed .pkl files.
    """
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    seed = 11
    for scenario_path in test_dirs:
        scenario_name = os.path.basename(scenario_path.rstrip('/'))
        print(f"Preprocessing scenario: {scenario_name}")

        # --- Preprocess Original Data ---
        original_transform = transforms.Compose([
            transforms.Resize((224, 224)),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ])
        original_test_set = datasets.ImageFolder(root=scenario_path, transform=original_transform)
        original_data = [item for item in original_test_set] # Store (image, label) tuples
        output_path_original = os.path.join(output_dir, f"{scenario_name}_original.pkl")
        with open(output_path_original, 'wb') as f:
            pickle.dump(original_data, f)
        print(f"Saved preprocessed original data to: {output_path_original}")

        # --- Preprocess Noisy Data ---
        min_std = 0.1
        max_std = 0.2
        random.seed(seed)
        random_std = random.uniform(min_std, max_std)
        gaussian_noise_transform = AddGaussianNoise(0., random_std)
        noisy_transform = transforms.Compose([
            transforms.Resize((224, 224)),
            transforms.ToTensor(),
            gaussian_noise_transform,
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ])
        noisy_test_set = datasets.ImageFolder(root=scenario_path, transform=noisy_transform)
        noisy_data = [item for item in noisy_test_set] # Store (image, label) tuples
        output_path_noisy = os.path.join(output_dir, f"{scenario_name}_noisy.pkl") # Modified noisy output name
        with open(output_path_noisy, 'wb') as f:
            pickle.dump(noisy_data, f)
        print(f"Saved preprocessed noisy data to: {output_path_noisy}")

        seed += 1

test_dirs = [           # test seniors
    "../realdata/NICO/public_dg_0416/train/dim",
    "../realdata/NICO/public_dg_0416/train/grass",
    "../realdata/NICO/public_dg_0416/train/outdoor",
    "../realdata/NICO/public_dg_0416/train/rock",
    "../realdata/NICO/public_dg_0416/train/water"
]

    # Preprocess the data (run this once)
preprocess_test_data(test_dirs)


In [ ]:
# Feature extraction Simple CNN
# UniformLoss: Encourages features to be uniformly distributed on the unit hypersphere
class UniformLoss(nn.Module):
    def __init__(self, t=2.0, lambda_u=0.01):
        """
        Args:
            t: Temperature parameter, adjusts the influence of distance
            lambda_u: Loss scaling factor
        """
        super(UniformLoss, self).__init__()
        self.t = t
        self.lambda_u = lambda_u

    def forward(self, features):
        """
        Args:
            features: [batch_size, feat_dim], feature vectors extracted by the network
        Returns:
            uniform_loss: Uniformity loss
        """
        # First, perform L2 normalization on the features to ensure they are on the unit hypersphere
        features = F.normalize(features, p=2, dim=1)

        # Calculate the cosine similarity matrix between features (direct inner product after normalization)
        sim_matrix = torch.mm(features, features.t())
        # Using the normalization property, the Euclidean distance can be written as: ||x-y||^2 = 2 - 2*(x·y)
        distances = 2 - 2 * sim_matrix

        n = features.size(0)
        # Exclude the diagonal (distance from itself is 0)
        mask = torch.eye(n, dtype=torch.bool, device=features.device)
        distances = distances[~mask].view(n, n - 1)

        # Calculate the uniformity loss: log(mean(exp(-t * distance)))
        loss = torch.log(torch.mean(torch.exp(-self.t * distances)))
        return self.lambda_u * loss

# Define a custom CNN model
class CustomCNN(nn.Module):
    def __init__(self, num_classes=60, feature_dim=10):
        super(CustomCNN, self).__init__()
        # Feature extraction part
        self.features = nn.Sequential(
            nn.Conv2d(3, 16, kernel_size=3, padding=1),  # Input: 3 channels
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Conv2d(16, 32, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Flatten()
        )
        # Fully connected layer after feature extraction
        self.feature_extractor = nn.Sequential(
            nn.Linear(64 * 28 * 28, 256),
            nn.ReLU(),
            nn.Linear(256, feature_dim)
        )
        # Classifier
        self.classifier = nn.Linear(feature_dim, num_classes)

    def forward(self, x):
        x = self.features(x)
        features = self.feature_extractor(x)  # Extract feature_dim dimensional features
        output = self.classifier(features)    # Get the classification output
        return output, features

# Initialize the model, loss functions, and optimizer
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = CustomCNN(num_classes=60, feature_dim=10).to(device)

criterion_ce = nn.CrossEntropyLoss().to(device)                      # Main classification loss
criterion_uniform = UniformLoss(t=2.0, lambda_u=0.003).to(device)  # Uniformity loss

# Optimizer updates both model parameters and learnable parameters in UniformLoss (if any)
optimizer = torch.optim.Adam([
    {'params': model.parameters()},
    {'params': criterion_uniform.parameters(), 'weight_decay': 5e-4}
], lr=0.001)

# Training loop
def train_model(dataloader, epochs=50):
    for epoch in range(epochs):
        model.train()
        running_loss = 0.0
        progress_bar = tqdm(dataloader, desc=f"Epoch {epoch+1}/{epochs}", leave=False)
        for images, labels in progress_bar:
            images = images.to(device)
            labels = labels.to(device)

            # Forward propagation
            outputs, features = model(images)

            # Calculate cross-entropy and uniformity loss
            loss_ce = criterion_ce(outputs, labels)
            loss_uniform = criterion_uniform(features)
            total_loss = loss_ce + loss_uniform

            # Backward propagation
            optimizer.zero_grad()
            total_loss.backward()
            optimizer.step()

            running_loss += total_loss.item()
            progress_bar.set_postfix(loss=running_loss / len(progress_bar))
        print(f"Epoch {epoch+1}, Total Loss: {running_loss / len(dataloader):.4f}")

# Assuming full_train_set is already defined
dataloader = DataLoader(full_train_set, batch_size=32, shuffle=True)

# Start training
train_model(dataloader, epochs=30)

def extract_features(model, dataloader):
    model.eval()
    features_all =[]
    labels_all =[]

    with torch.no_grad():
        for images, labels in dataloader:
            images = images.to(device)
            _, features = model(images)
            features_all.append(features)
            labels_all.append(labels)

    features_all = torch.cat(features_all, dim=0)
    labels_all = torch.cat(labels_all, dim=0)
    np.save('supervised_features.npy', features_all.cpu().numpy())
    return features_all, labels_all
# Extract features
features, labels = extract_features(model, dataloader)
print(np.shape(features))

In [ ]:
## Using a pre-trained ResNet50 feature extraction CNN -
class FeatureExtractor(nn.Module):
    def __init__(self, feature_dim=10):
        super(FeatureExtractor, self).__init__()
        # Load the pre-trained ResNet50 model (IMAGENET1K_V2)
        self.resnet50 = models.resnet50(weights=models.ResNet50_Weights.IMAGENET1K_V2)

        # Freeze all convolutional layer parameters except for layer4
        for name, param in self.resnet50.named_parameters():
            if not name.startswith('layer4'):
                param.requires_grad = False

        # Replace the last fully connected layer of ResNet50 to extract features of the specified dimension
        num_ftrs = self.resnet50.fc.in_features
        self.feature_extractor = nn.Linear(num_ftrs, feature_dim)
        self.classifier = nn.Linear(feature_dim, 60) # Assuming the number of classes is 60

    def forward(self, x):
        # Pass through the convolutional layers of ResNet50
        x = self.resnet50.conv1(x)
        x = self.resnet50.bn1(x)
        x = self.resnet50.relu(x)
        x = self.resnet50.maxpool(x)

        x = self.resnet50.layer1(x)
        x = self.resnet50.layer2(x)
        x = self.resnet50.layer3(x)
        x = self.resnet50.layer4(x)

        x = self.resnet50.avgpool(x)
        x = torch.flatten(x, 1)

        # Pass through the custom feature extraction layer
        features = self.feature_extractor(x)
        output = self.classifier(features) # Add a classifier if needed
        return output, features

# Initialize the model, loss function, and optimizer
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = FeatureExtractor(feature_dim=10).to(device)

criterion_ce = nn.CrossEntropyLoss().to(device)  # Main classification loss

# Optimize only the parameters of layer4 and the custom feature extraction layer and classifier
params_to_optimize = []
for name, param in model.named_parameters():
    if param.requires_grad:
        params_to_optimize.append(param)

optimizer = torch.optim.Adam(params_to_optimize, lr=0.001)

# Training loop
def train_model(dataloader, epochs=50):
    for epoch in range(epochs):
        model.train()
        running_loss = 0.0
        progress_bar = tqdm(dataloader, desc=f"Epoch {epoch+1}/{epochs}", leave=False)
        for images, labels in progress_bar:
            images = images.to(device)
            labels = labels.to(device)

            # Forward propagation
            outputs, features = model(images)

            # Calculate cross-entropy loss
            loss_ce = criterion_ce(outputs, labels)
            total_loss = loss_ce

            # Backward propagation
            optimizer.zero_grad()
            total_loss.backward()
            optimizer.step()

            running_loss += total_loss.item()
            progress_bar.set_postfix(loss=running_loss / len(progress_bar))
        print(f"Epoch {epoch+1}, Total Loss: {running_loss / len(dataloader):.4f}")

# Start training
dataloader = DataLoader(full_train_set, batch_size=64, shuffle=True)
train_model(dataloader, epochs=30)

# Feature extraction part remains unchanged
def extract_features(model, dataloader):
    model.eval()
    features_all =[]
    labels_all =[]

    with torch.no_grad():
        for images, labels in dataloader:
            images = images.to(device)
            _, features = model(images)
            features_all.append(features.cpu().numpy())
            labels_all.append(labels.cpu().numpy())

    features_all = np.concatenate(features_all, axis=0)
    labels_all = np.concatenate(labels_all, axis=0)
    np.save('supervised_features.npy', features_all)
    return features_all, labels_all

# Extract features
features, labels = extract_features(model, dataloader)
print(np.shape(features))

In [ ]:
## Subsampling USSP IBOSS LowCon SRS and Baseline
import numpy as np
UD=np.load('UD_2000_10.npy')    
LHD=np.load('LHD_2000_10.npy')
NICO_autumn_feature=np.load('supervised_features.npy')    
print(np.shape(UD))
n_samples = NICO_autumn_feature.shape[0]

N=2000
_,NICO_autumn_subsample_USSP_index=USSP.USSP(NICO_autumn_feature,UD)
NICO_autumn_subsample_IBOSS_index,_=IBOSS.IBOSS(NICO_autumn_feature, N)
NICO_autumn_subsample_Lowcon_index=LowCon.LowCon_without_rep(NICO_autumn_feature, LHD)
NICO_autumn_subsample_srs_index=np.random.choice(n_samples, size=N, replace=False)
NICO_autumn_subsample_all_index=arr = np.arange(n_samples)

In [ ]:
## Baseline training
# filename = "NICO_autumn_enhanced_dataset.pkl"
# with open(filename, 'rb') as f:
#     full_train_set = pickle.load(f)

all_results = {}
samples_all={"all": NICO_autumn_subsample_all_index}    # Index all
for subset_name, indices in samples_all.items():
    # Train
    model_path = f"best_model_{subset_name}.pth"  
    best_val_acc, log = NICO_ResNet50_TrainTest.train_model(full_train_set,indices, model_path,64,10)
    
    # Test 
    test_accs = NICO_ResNet50_TrainTest.test_scenarios(model_path)
    
    # Result
    all_results[subset_name] = {
        'val_acc': best_val_acc,
        'test_accs': test_accs,
        'log': log
    }


for subset_name, metrics in all_results.items():
    avg_test = np.mean(list(metrics['test_accs'].values()))
    print(f"{subset_name}: ValAcc={metrics['val_acc']:.1f}% | AvgTestAcc={avg_test:.1f}%")

In [ ]:
## USSP IBOSS LowCon SRS training
all_results = {}
subsamples = {
        # "USSP": NICO_autumn_subsample_USSP_index,
        "IBOSS": NICO_autumn_subsample_IBOSS_index,
        "Lowcon": NICO_autumn_subsample_Lowcon_index,
        "srs": NICO_autumn_subsample_srs_index,  # random index
}   
for subset_name, indices in subsamples.items():
    # train 
    model_path = f"best_model_{subset_name}.pth"  
    best_val_acc, log = NICO_ResNet50_TrainTest.train_model(full_train_set,indices, model_path,32,10,0.0001)
    
    # test
    test_accs = NICO_ResNet50_TrainTest.test_scenarios(model_path)
    
    # result
    all_results[subset_name] = {
        'val_acc': best_val_acc,
        'test_accs': test_accs,
        'log': log
    }

# result analysis
print("\nAccuracy results:")
for subset_name, metrics in all_results.items():
    avg_test = np.mean(list(metrics['test_accs'].values()))
    print(f"{subset_name}: ValAcc={metrics['val_acc']:.1f}% | AvgTestAcc={avg_test:.1f}%")

In [ ]:
## USSP tune
import NICO_ResNet50_TrainTest
reload(NICO_ResNet50_TrainTest)
import USSP
reload(USSP)
import numpy as np

all_results = {}
num_repetitions = 20

subsamples_to_run = {
    "USSP": None, # We will generate the indices inside the loop
}

for subset_name in subsamples_to_run:
    all_val_accs = []
    all_test_accs_list = []
    all_logs = []

    print(f"\nRunning {subset_name} for {num_repetitions} repetitions...")

    for i in range(num_repetitions):
        print(f"\n--- Repetition {i+1}/{num_repetitions} for {subset_name} ---")
        # Generate new USSP indices for each repetition
        _, NICO_autumn_subsample_USSP_index = USSP.USSP(NICO_autumn_feature, UD)
        indices = NICO_autumn_subsample_USSP_index

        # train
        model_path = f"best_model_{subset_name}_rep{i+1}.pth"
        best_val_acc, log = NICO_ResNet50_TrainTest.train_model(full_train_set, indices, model_path, 16, 30)
        all_val_accs.append(best_val_acc)
        all_logs.append(log)

        # test
        test_accs = NICO_ResNet50_TrainTest.test_scenarios(model_path)
        all_test_accs_list.append(test_accs)

    # Calculate average validation accuracy
    avg_val_acc = np.mean(all_val_accs)

    # Calculate average test accuracy across all scenarios
    all_scenario_names = set()
    for test_accs in all_test_accs_list:
        all_scenario_names.update(test_accs.keys())

    avg_test_accs = {}
    for scenario in all_scenario_names:
        scenario_accuracies = [accs.get(scenario, 0) for accs in all_test_accs_list]
        avg_test_accs[scenario] = np.mean(scenario_accuracies)

    # Store the average results
    all_results[subset_name] = {
        'avg_val_acc': avg_val_acc,
        'avg_test_accs': avg_test_accs,
        'all_logs': all_logs # Optionally store all logs
    }

# Result print
print("\nAverage accuracy results across 20 repetitions:")
for subset_name, metrics in all_results.items():
    avg_test_overall = np.mean(list(metrics['avg_test_accs'].values()))
    print(f"{subset_name}: AvgValAcc={metrics['avg_val_acc']:.1f}% | AvgTestAcc={avg_test_overall:.1f}%")

    print(f"\nDetailed average test accuracy per scenario for {subset_name}:")
    for scenario, avg_acc in metrics['avg_test_accs'].items():
        print(f"  {scenario}: {avg_acc:.1f}%")

In [ ]:
## Save the results to matlab to plot
# subset_names = list(all_results.keys())
subset_names=['all','Lowcon','IBOSS','srs','USSP']
# scenario_names =['dim','dim_noisy','grass','grass_noisy','outdoor','outdoor_noisy','rock','rock_noisy','water','water_noisy']   
scenario_names = list(list(all_results.values())[0]['test_accs'].keys())    
acc_matrix = np.zeros((len(all_results), len(scenario_names)))

# fix the matrix
for row_idx, subset_name in enumerate(subset_names):
    test_accs = all_results[subset_name]['test_accs']
    row_data = [test_accs[scenario] for scenario in scenario_names]
    acc_matrix[row_idx, :] = row_data

# Construct a dictionary containing the matrix and labels
save_dict = {
    'acc_matrix': acc_matrix,            # accuracy matrix 
    'scenario_names': scenario_names,    # col labels
    'subset_names': subset_names         # row labels
}

# save to .mat file
savemat('cross_subset_accuracy.mat', save_dict)

In [ ]:
## USSP with different subsample numbers
import numpy as np
import USSP
reload(USSP)

NICO_autumn_feature=np.load('supervised_features.npy')
n_values = [200, 500, 800, 1000, 1500, 2000, 2500, 3000, 4000]
results = []

for n in n_values:
    filename_ud = f'UD_{n}_10.npy'
    try:
        UD = np.load(filename_ud)
        _, NICO_autumn_subsample_USSP_index = USSP.USSP(NICO_autumn_feature, UD)
        results.append(NICO_autumn_subsample_USSP_index)
        print(f"Processed n = {n}, loaded {filename_ud}, and stored the index.")
    except FileNotFoundError:
        print(f"Error: File {filename_ud} not found.")
    except Exception as e:
        print(f"An error occurred while processing n = {n}: {e}")

# The results list contains the NICO_autumn_subsample_USSP_index obtained after running USSP.USSP for each n value
print("\nResults (list of NICO_autumn_subsample_USSP_index for each n):")
for i, result in enumerate(results):
    print(f"n = {n_values[i]}: {result.shape if isinstance(result, np.ndarray) else result}")

import NICO_ResNet50_TrainTest
reload(NICO_ResNet50_TrainTest)
all_results_2 = {}
for i, indices in enumerate(results):
    n_value = n_values[i]
    subset_name = f"USSP_n_{n_value}"
    print(f"\nTraining and testing model for {subset_name} with {len(indices)} indices.")

    # Training phase
    model_path = f"best_model_{subset_name}.pth"  # Name the model according to the subset name
    best_val_acc, log = NICO_ResNet50_TrainTest.train_model(full_train_set, indices, model_path, 64, 30,0.001) # Reduce epochs for demonstration

    # Testing phase
    test_accs = NICO_ResNet50_TrainTest.test_scenarios(model_path)

    # Record the results (using readable names)
    all_results_2[subset_name] = {
        'val_acc': best_val_acc,
        'test_accs': test_accs,
        'log': log
    }

In [ ]:
## Training the esNet50
import NICO_ResNet50_TrainTest
reload(NICO_ResNet50_TrainTest)
all_results_2 = {}
for i, indices in enumerate(results):
    n_value = n_values[i]
    subset_name = f"USSP_n_{n_value}"
    print(f"\nTraining and testing model for {subset_name} with {len(indices)} indices.")

    # Train
    model_path = f"best_model_{subset_name}.pth"  
    best_val_acc, log = NICO_ResNet50_TrainTest.train_model(full_train_set, indices, model_path,  32, 30,0.0005) 

    # Test
    test_accs = NICO_ResNet50_TrainTest.test_scenarios(model_path)

    # Records
    all_results_2[subset_name] = {
        'val_acc': best_val_acc,
        'test_accs': test_accs,
        'log': log
    }

In [ ]:
# Result analysis (comparison with readable names)
print("\nPerformance comparison across subsets:")
for subset_name, metrics in all_results_2.items():
    avg_test = np.mean(list(metrics['test_accs'].values()))
    print(f"{subset_name}: ValAcc={metrics['val_acc']:.1f}% | AvgTestAcc={avg_test:.1f}%")

# Output as a table
print("\nTest Accuracy per Scenario:")

# Get the names of all test scenarios as column names
all_scenario_names = set()
for metrics in all_results_2.values():
    all_scenario_names.update(metrics['test_accs'].keys())

def custom_sort_key(scenario_name):
    if scenario_name.endswith("_original"):
        return (scenario_name[:-9], 0)  # Original ones come first (0)
    elif scenario_name.endswith("_noisy"):
        return (scenario_name[:-6], 1)  # Noisy ones come last (1)
    else:
        return (scenario_name, 2)  # Other cases in the middle (2)

scenario_names = sorted(list(all_scenario_names), key=custom_sort_key)

# Print the table header
header = ["Index"] + [name for name in scenario_names]
print("| " + " | ".join(header) + " |")
print("|" + "---|" * (len(header)))

# Print the results for each row
for subset_name, metrics in all_results_2.items():
    row = [subset_name.split('_')[-1]]  # Use the n value as the Index
    for scenario in scenario_names:
        accuracy = metrics['test_accs'].get(scenario, "N/A")
        row.append(f"{accuracy:.1f}%" if isinstance(accuracy, (int, float)) else accuracy)
    print("| " + " | ".join(row) + " |")

In [ ]:
## Running time compare
import numpy as np
import USSP
import IBOSS
import LowCon
import time
import os

# Define the value list for d and n
d_values = 10
n_values = [200, 500, 800, 1000, 1500, 2000, 2500, 3000, 4000]
totalnumber = 27282

# Ensure the directory for saving npy files is the same as the current script, or provide the correct path
output_dir = "."  # Assuming npy files are saved in the current directory

# Dictionary to store all test results (modified to have method as the outer key)
all_results = {
    'USSP': {},
    'IBOSS': {},
    'Lowcon': {},
    'SRS': {}
}

# Generate normally distributed points K
K = np.random.randn(totalnumber, d_values)

for N in n_values:
    filename = f"lhd_design_d{d_values}_n{N}.npy"
    filepath = os.path.join(output_dir, filename)
    try:
        LHD = np.load(filepath)
        UD = LHD.copy()  # Assign the same data to LHD and UD
        print(f"\n--- d={d_values}, N={N}, totalnumber={totalnumber} ---")

        # Time USSP
        start_time = time.time()
        _, _ = USSP.USSP(K, UD)
        end_time = time.time()
        all_results['USSP'][N] = f"{end_time - start_time:.6f}"

        # Time IBOSS
        start_time = time.time()
        _, _ = IBOSS.IBOSS(K, N)
        end_time = time.time()
        all_results['IBOSS'][N] = f"{end_time - start_time:.6f}"

        # Time Lowcon
        start_time = time.time()
        _ = LowCon.LowCon(K, LHD)
        end_time = time.time()
        all_results['Lowcon'][N] = f"{end_time - start_time:.6f}"

        # Time srs
        start_time = time.time()
        _ = np.random.choice(totalnumber, size=N, replace=False)
        end_time = time.time()
        all_results['SRS'][N] = f"{end_time - start_time:.6f}"

    except FileNotFoundError:
        print(f"Warning: File {filepath} not found, skipping the combination of d={d_values}, N={N}.")

# Output results table
print("\n--- Execution Time of Sampling Methods (Unit: Seconds) ---")
header = ["n_values"] + list(all_results.keys())
print("| " + " | ".join(header) + " |")
print("|" + "---|" * (len(header)))

for n_val in n_values:
    row = [str(n_val)]
    for method in all_results.keys():
        row.append(all_results[method].get(n_val, "N/A"))
    print("| " + " | ".join(row) + " |")

print("\nTiming of all functions completed and output as a table.")